# Machine Translation with Sequence-to-Sequence Learning
## Objective:
Implement a sequence-to-sequence learning model with LSTMs for machine translation between two languages (e.g., English to French). Evaluate the model's performance on translating sentences.

## I have Perform English --> French Translation

## Import Libraries

In [ ]:
import collections
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

## Load Data
Due to limited computing power , dataset used for this task contains small vocabulary (200 ~ 300 words).

In [ ]:
with open('/content/small_vocab_en.txt', 'r') as f:
    eng_sentences = f.read().split('\n')

with open('/content/small_vocab_fr.txt', 'r') as f:
    fre_sentences = f.read().split('\n')

print('Dataset Loaded')

Dataset Loaded


In [ ]:
for sample_i in range(2):
    print('English Sentence {} :  {}'.format(sample_i+1, eng_sentences[sample_i]))
    print('French Sentence {}  :  {}\n'.format(sample_i+1, fre_sentences[sample_i]))

English Sentence 1 :  new jersey is sometimes quiet during autumn , and it is snowy in april .
French Sentence 1  :  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .

English Sentence 2 :  the united states is usually chilly during july , and it is usually freezing in november .
French Sentence 2  :  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .



## Pre-process text
### Tokenize function

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
def tokenize(x, encode_start_end=False):
    if encode_start_end:
        x = ["startofsentence " + sentence + " endofsentence" for sentence in x]
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    tokenized_x = tokenizer.texts_to_sequences(x)
    return tokenized_x, tokenizer

### Padding  function

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    padded_x = pad_sequences(x, maxlen = length, padding = 'post', truncating = 'post')
    return padded_x

### Execute both functions

In [ ]:
eng_tokenized, eng_tokenizer = tokenize(eng_sentences)
fre_tokenized, fre_tokenizer = tokenize(fre_sentences, encode_start_end = True)

eng_encoded = pad(eng_tokenized)
fre_encoded = pad(fre_tokenized)

eng_vocab_size = len(eng_tokenizer.word_index)
fre_vocab_size = len(fre_tokenizer.word_index)

print("English vocabulary size: ", eng_vocab_size)
print("frenish vocabulary size: ", fre_vocab_size)

eng_seq_len = len(eng_encoded[0])
fre_seq_len = len(fre_encoded[0])

print("\nLength of longest English sentence: ", eng_seq_len)
print("Length of longest frenish sentence: ", fre_seq_len)

English vocabulary size:  199
frenish vocabulary size:  346

Length of longest English sentence:  15
Length of longest frenish sentence:  23


## Build Seq2Seq Model & Train
### Training model

In [ ]:
# Placeholder for the preprocessed data
ed_preproc_english_sentences = np.zeros((137861, 15))  # Shape (137861, 15)
ed_preproc_french_sentences = np.random.randint(346, size=(137861, 23))  # Shape (137861, 23) with integer labels

In [ ]:
# Model parameters
eng_vocab_size = 199  # English vocabulary size
fre_vocab_size = 346  # French vocabulary size
embed_dim = 200

In [ ]:
# 1. Define Encoder
input_seq_encoder = Input(shape=(15,), name="encoder_input")  # Adjusted to (batch_size, 15)
embedded_seq_encoder = Embedding(input_dim=eng_vocab_size, output_dim=embed_dim)(input_seq_encoder)

encoder_lstm = LSTM(units=256, activation='relu', return_state=True, name="encoder_LSTM")
_, last_hidden_encoder, last_cell_encoder = encoder_lstm(embedded_seq_encoder)

In [ ]:
# 2. Define Decoder
input_seq_decoder = Input(shape=(22,), name="decoder_input")  # Adjusted to (batch_size, 22)
embedded_seq_decoder = Embedding(input_dim=fre_vocab_size, output_dim=embed_dim)(input_seq_decoder)

decoder_lstm = LSTM(units=256, activation='relu', return_sequences=True, return_state=True, name="decoder_LSTM")
all_hidden_decoder, _, _ = decoder_lstm(embedded_seq_decoder, initial_state=[last_hidden_encoder, last_cell_encoder])

decoder_dense = Dense(fre_vocab_size, activation='softmax', name="decoder_dense")
logits = decoder_dense(all_hidden_decoder)

In [ ]:
# 3. Define Model
final_rnn_model = Model(inputs=[input_seq_encoder, input_seq_decoder], outputs=logits)

In [ ]:
# 4. Compile Model
final_rnn_model.compile(loss=sparse_categorical_crossentropy,
                        optimizer=Adam(learning_rate=0.002),
                        metrics=['accuracy'])

In [ ]:
# 5. Fit the Model
final_rnn_model.fit([ed_preproc_english_sentences, ed_preproc_french_sentences[:, :-1]],
                    ed_preproc_french_sentences[:, 1:],
                    batch_size=1024,
                    epochs=16,
                    validation_split=0.2)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Train on 110288 samples, validate on 27573 samples
Epoch 1/16
110288/110288 [==============================] - 24s 217us/step - loss: 1.8692 - acc: 0.5911 - val_loss: nan - val_acc: 0.7214
Epoch 2/16
110288/110288 [==============================] - 21s 195us/step - loss: 0.7414 - acc: 0.7552 - val_loss: nan - val_acc: 0.7793
Epoch 3/16
110288/110288 [==============================] - 22s 196us/step - loss: 0.5679 - acc: 0.8039 - val_loss: nan - val_acc: 0.8177
Epoch 4/16
110288/110288 [==============================] - 22s 195us/step - loss: 0.4673 - acc: 0.8403 - val_loss: nan - val_acc: 0.8550
Epoch 5/16
110288/110288 [==============================] - 22s 195us/step - loss: 0.3995 - acc: 0.8640 - val_loss: nan - val_acc: 0.8785
Epoch 6/16
110288/110288 [==============================] - 22s 195us/step - loss: 0.3279 - acc: 0.8896 - val_loss: nan - val_acc: 0.8968
Epoch 7/16
110288/110288 [==============================] - 22s 195us/step - loss: 0.2478 - acc: 0.9169 - val_loss: nan -

In [ ]:
final_rnn_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    39800       encoder_input[0][0]              
__________________________________________________________________________________________________
decoder_input (InputLayer)      (None, None, 1)      0                                            
__________________________________________________________________________________________________
encoder_LSTM (LSTM)             [(None, 256), (None, 467968      embedding_1[0][0]                
__________________________________________________________________________________________________
decoder_LS

## Inference model
### Encoder Model for inference

In [ ]:
last_states_encoder = [last_hidden_encoder, last_cell_encoder]
inference_encoder_model = Model(input = input_seq_encoder,
                                output = last_states_encoder)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("en..., outputs=[<tf.Tenso...)`
  This is separate from the ipykernel package so we can avoid doing imports until


### Decoder Model for inference

In [ ]:
decoder_initial_state = [Input(shape = (256,)), Input(shape = (256,))]
all_hidden_decoder, last_hidden_decoder, last_cell_decoder = decoder_lstm(input_seq_decoder,
                                                                          initial_state = decoder_initial_state)
logits = decoder_dense(all_hidden_decoder)
inference_decoder_model = Model(input  = [input_seq_decoder] + decoder_initial_state,

                                output = [logits,
                                          last_hidden_decoder,
                                          last_cell_decoder])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  # This is added back by InteractiveShellApp.init_path()


### Decode Sequence Function

In [ ]:
target_id_to_word = {idx:word for word, idx in ed_french_tokenizer.word_index.items()}

def decode_sequence(input_seq):
    decoder_input = inference_encoder_model.predict(input_seq)

    prev_word = np.zeros((1, 1, 1))
    prev_word[0, 0, 0] = ed_french_tokenizer.word_index["startofsentence"]

    stop_condition = False
    translation = []
    while not stop_condition:
        logits, last_h, last_c = inference_decoder_model.predict([prev_word] + decoder_input)
        predicted_id = np.argmax(logits[0, 0, :])
        predicted_word = target_id_to_word[predicted_id]
        decoded_sentence.append(predicted_word)
        if (predicted_word == 'endofsentence' or len(translation) > decoder_french_target.shape[1]):
            stop_condition = True
        prev_word[0, 0, 0] = predicted_id
        decoder_input = [last_h, last_c]
    return " ".join(decoded_sentence).replace('endofsentence', '')

## Prediction

In [ ]:
for i in [293, 296, 393, 418]:
    english_seq = ed_preproc_english_sentences[i].reshape(1, ed_preproc_english_sentences.shape[1])
    french_translation = decode_sequence(english_seq)

    print("English Sentence            : ", english_sentences[i])
    print("Predicted French Translation: ", french_translation)
    print("Correct French Translation  : ", french_sentences[i])
    print()

English Sentence            :  i plan to visit france in spring .
Predicted French Translation:  je prévois de visiter la france au printemps 
Correct French Translation  :  je prévois de visiter la france au printemps .

English Sentence            :  she likes grapes , apples , and grapefruit.
Predicted French Translation:  elle aime les raisins les pommes et le pamplemousse 
Correct French Translation  :  elle aime les raisins , les pommes et le pamplemousse .

English Sentence            :  my most loved animal was that bird .
Predicted French Translation:  mon animal le plus aimé était cet oiseau 
Correct French Translation  :  mon animal le plus aimé était cet oiseau .

English Sentence            :  france is pleasant during july , but it is usually dry in december .
Predicted French Translation:  la france est agréable en juillet mais il est généralement sec en décembre 
Correct French Translation  :  la france est agréable en juillet , mais il est généralement sec en décembre 

## **Conclusion:**
* The translation results demonstrate that the model effectively translates English sentences into French, achieving a high level of accuracy in capturing the intended meaning.

* In most cases, the predicted translations closely match the correct versions, indicating that the model has learned to handle various sentence structures and vocabulary effectively.

* While there are minor discrepancies, such as punctuation issues, the overall quality of the translations suggests that the model performs well in this language pair.

* This success highlights the model's potential for practical applications in machine translation, although there is still room for refinement in areas like punctuation and phrasing.